In [ ]:
!pip install pypdf langchain_openai langchain_community faiss-cpu --quiet

In [5]:
import os
from google.colab import userdata

In [ ]:
#set openai key
os.environ["OPENAI_API_KEY"] = userdata.get("openai")

In [ ]:
#import LLM
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4")
llm.invoke("Hi")

In [7]:
from langchain_community.document_loaders import PyPDFLoader

In [9]:
#load and parse PDF
loader = PyPDFLoader("") #paste path to pdf
pages = loader.load_and_split()

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [ ]:
#load to knowledge base
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())

faiss_index.save_local('document')

In [14]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI

In [ ]:
#load the saved embeddings
embeddings = OpenAIEmbeddings()

vectors = FAISS.load_local('document', embeddings, allow_dangerous_deserialization=True)

#create a conversational retrieval chain
qa = ConversationalRetrievalChain.from_llm(
    llm = OpenAI(),
    retriever = vectors.as_retriever()
)

chat_history = []

def rag(query):
  response = qa({"question": query, "chat_history": chat_history})
  chat_history.append((query, response['answer']))

  return response['answer'].strip()

In [ ]:
rag("") #ask a question